### Description

In data-exploration-1, we did some initial data exploration to find out what kind of data we had, and to see if we needed to "clean" the data to deal with missing or null values. Now that we have a better handle on the content of the data, we can merge the data from the various files into a dataframe so that we can fully compare the two restaurant reservation systems.


In [1]:
#setup
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
%matplotlib inline

#for visualization of missing data
import missingno

aReserveDF = pd.read_csv('air_reserve.csv', parse_dates = ['visit_datetime', 'reserve_datetime']) 
aVisitDF = pd.read_csv('air_visit_data.csv', parse_dates = ['visit_date']) 
aStoreDF = pd.read_csv('air_store_info.csv')

hReserveDF = pd.read_csv('hpg_reserve.csv', parse_dates = ['visit_datetime', 'reserve_datetime']) 
hStoreDF = pd.read_csv('hpg_store_info.csv') 

dateInfoDF = pd.read_csv('date_info.csv', parse_dates = ['calendar_date'])
sampleSubmissionDF = pd.read_csv('sample_submission.csv') 
storeIdRelationDF = pd.read_csv('store_id_relation.csv') 

### Merging DataFrames

Next, we'll merge data from the air Reservation System with data from the HPG Reservation System.
We'll borrow the code for merging from http://www.kaggle.com/oxanozaep/eda-and-regression-analysis

In [2]:
#parse the visit_datetime and reserve_datetime into separate years, months, days
#then, drop the now unnecessary visit_datetime and reserve_datetime categories
hReserveDF['visit_year'] = hReserveDF['visit_datetime'].dt.year
hReserveDF['visit_month'] = hReserveDF['visit_datetime'].dt.month
hReserveDF['visit_day'] = hReserveDF['visit_datetime'].dt.day
hReserveDF['reserve_year'] = hReserveDF['reserve_datetime'].dt.year#
hReserveDF['reserve_month'] = hReserveDF['reserve_datetime'].dt.month
hReserveDF['reserve_day'] = hReserveDF['reserve_datetime'].dt.day
hReserveDF.drop(['visit_datetime','reserve_datetime'], axis=1, inplace=True)

hReserveDF = hReserveDF.groupby(['hpg_store_id', 'visit_year', 'visit_month',\
                                   'visit_day','reserve_year','reserve_month','reserve_day'], as_index=False).sum()

aReserveDF['visit_year'] = aReserveDF['visit_datetime'].dt.year
aReserveDF['visit_month'] = aReserveDF['visit_datetime'].dt.month
aReserveDF['visit_day'] = aReserveDF['visit_datetime'].dt.day
aReserveDF['reserve_year'] = aReserveDF['reserve_datetime'].dt.year
aReserveDF['reserve_month'] = aReserveDF['reserve_datetime'].dt.month
aReserveDF['reserve_day'] = aReserveDF['reserve_datetime'].dt.day
aReserveDF.drop(['visit_datetime','reserve_datetime'], axis=1, inplace=True)

dateInfoDF['calendar_year'] = dateInfoDF['calendar_date'].dt.year
dateInfoDF['calendar_month'] = dateInfoDF['calendar_date'].dt.month
dateInfoDF['calendar_day'] = dateInfoDF['calendar_date'].dt.day
dateInfoDF.drop(['calendar_date'], axis=1, inplace=True)

aVisitDF['visit_year'] = aVisitDF['visit_date'].dt.year
aVisitDF['visit_month'] = aVisitDF['visit_date'].dt.month
aVisitDF['visit_day'] = aVisitDF['visit_date'].dt.day
aVisitDF.drop(['visit_date'], axis=1, inplace=True)

hReserveDF = pd.merge(hReserveDF, storeIdRelationDF, on='hpg_store_id', how='inner')
hReserveDF.drop(['hpg_store_id'], axis=1, inplace=True)

aReserveDF = pd.concat([aReserveDF, hReserveDF])

aReserveDF.shape

(118910, 8)

### Merger Steps

Note that the reserve_datetime refers to the time that a potential customer visited the website and made a reservation, and NOT the actual day the customer visited a potential restaurant. Also, reserve_visitors shows the number of visitors that the reservation was made from.

visit_datetime is the actual day that the reservation is being made for.

We originally had 92,378 reservations made through the Air Reservation Sytem, and 2,000,320 reservations made through the HPG Reservation System.

There were only 150 correlations between Air Reservation Sytem IDs and HPG Reservation System IDs.
We only have real visitor data for restaurants that have an Air Reservation System ID.
So, after correlations, we only have 26,532 reservations made through the HPG Reservation System that have a correlated Air Reservation System ID.

So, we have 2,000,320 - 92,378  = 118,910 reservations total


### Summing the number of reservations for a particular store on a particular day

The line of code that follows sums the number of reservations for a particular air_store_id for a particular ID. This creates a dataframe where we will be able to compare daily reservation numbers for a restaurant with actual visitors for a restaurant.

The one problem with this approach is that if only some people make multiple reservations at once for a restaurant (e.g., they decide to book a 5 PM reservation for 10 guests and then another 6 PM reservation for the same 10 guests), then the number of reservations is overestimated compared to the actual demand. 



In [3]:
aReserveDF = aReserveDF.groupby(['air_store_id', 'visit_year', 'visit_month','visit_day'],\
                         as_index=False).sum().drop(['reserve_day','reserve_month','reserve_year'], axis=1)

aReserveDF.shape

(42193, 5)

After performing the above sum, we find that we now have 42,193 rows of data - for each of the IDs, we have data on how many visitors are expected on which day.

The next steps adds to the aReserveDF the general store information (store description).

Also, another DF called trainDF is created, which includes as a column "visitors," which tells the actual number of visits to a restaurant. 

In [4]:
aReserveDF = pd.merge(aReserveDF, dateInfoDF, left_on=['visit_year','visit_month','visit_day'], right_on=['calendar_year','calendar_month','calendar_day'], how='left')
aReserveDF.drop(['calendar_year','calendar_month','calendar_day'], axis=1, inplace=True)

aReserveDF = pd.merge(aReserveDF, aStoreDF, on='air_store_id', how='left')

trainDF = pd.merge(aReserveDF, aVisitDF, on=['air_store_id','visit_year','visit_month','visit_day'], how='left')

In [5]:
aReserveDF.head()

,air_store_id,visit_year,visit_month,visit_day,reserve_visitors,day_of_week,holiday_flg,air_genre_name,air_area_name,latitude,longitude
0,air_00a91d42b08b08d9,2016,1,14,2,Thursday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595
1,air_00a91d42b08b08d9,2016,1,15,4,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595
2,air_00a91d42b08b08d9,2016,1,16,2,Saturday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595
3,air_00a91d42b08b08d9,2016,1,22,2,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595
4,air_00a91d42b08b08d9,2016,1,29,5,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595


In [6]:
trainDF.head()

,air_store_id,visit_year,visit_month,visit_day,reserve_visitors,day_of_week,holiday_flg,air_genre_name,air_area_name,latitude,longitude,visitors
0,air_00a91d42b08b08d9,2016,1,14,2,Thursday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,NaN
1,air_00a91d42b08b08d9,2016,1,15,4,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,NaN
2,air_00a91d42b08b08d9,2016,1,16,2,Saturday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,NaN
3,air_00a91d42b08b08d9,2016,1,22,2,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,NaN
4,air_00a91d42b08b08d9,2016,1,29,5,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,NaN


From the above, we see that there are some NaN values in the trainDF dataframe. Let's check how many.

In [7]:
trainDF.isnull().sum()

air_store_id           0
visit_year             0
visit_month            0
visit_day              0
reserve_visitors       0
day_of_week            0
holiday_flg            0
air_genre_name         0
air_area_name          0
latitude               0
longitude              0
visitors            6495
dtype: int64

### Unfulfilled reservations

It seems that the 6,495 NaN values in the visitor column show cases where a reservation was made for a specific restaurant on a specific date, but that there were no actual visitors.

Let's also look at a few statistics for the trainDF.

In [8]:
trainDF.describe(include = ['O'])

,air_store_id,day_of_week,air_genre_name,air_area_name
count,42193,42193,42193,42193
unique,333,7,14,72
top,air_6b15edd1b4fbb96a,Friday,Izakaya,Fukuoka-ken Fukuoka-shi Daimyō
freq,384,8302,15717,3302


Our trainDF dataframe shows that the 42,193 reservations were made at 333 unique restaurants. 

The most popular restaurant received 384 reservations.

Also, the most popular day for a reservation was Friday.